In [37]:
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:.3f}'.format

Autos = pd.read_csv('autos.csv', encoding = 'Latin-1')

#print(Autos.head())
#print(Autos.info())

original_columns = Autos.columns # I use this to put the name of the columns in an array

#print(original_columns)

Autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test','vehicle_type', 'registration_year', 'gear_box', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand','unrepaired_damage', 'ad_created', 'no_of_pictures', 'postal_code',
       'last_seen']

#print(Autos.head())

#print(Autos.describe(include='all'))  # to explore the data, the keyword argument is so that it will return both numeric and string columns

#the price and odometer columns are numeric values stored as text, we need to convert them to numeric

Autos["price"] = Autos["price"].str.replace('$','')
Autos["price"] = Autos["price"].str.replace(',','')
Autos["price"] = Autos["price"].astype(float)

#print(Autos["price"].head(20))

#Now we will convert the odometer column t numeric values

#print(Autos["odometer"].unique()) to print all the unique values in the column


Autos["odometer"] = Autos["odometer"].str.replace('km','')
Autos["odometer"] = Autos["odometer"].str.replace(',','')
Autos["odometer"] = Autos["odometer"].astype(int)

Autos.rename({'odometer':'odometer_km'}, axis = 1, inplace = True)  #To rename the column
#print(Autos['odometer_km'].head())

#Exploring the date in odometer column

#print(Autos["odometer_km"].unique().shape)  #This will give the number of unique values in that column
#print(Autos["odometer_km"].describe())   # This gives info on the mean, max, min etc

#5,000km in the odometer column seems pretty small compared to the other values but I will check out the unique value counts to the frequency of the value

#print(Autos["odometer_km"].value_counts().shape)  # To first see if there are a lot of value counts, if there are a lot, I will probably use head to check out the first few ones or use sort_index() to change to ascending order or something.
#print(Autos["odometer_km"].value_counts())

#So, from checking out the values, 5,000 seems to have about 1000 entries, so it is likely not a mistake.

#Exploring the date in price column

#print(Autos['price'].describe()) # to show us mean, max, min etc

#The max value of this column is way higher than the other values, but lets investigate further.

#print(Autos['price'].unique().shape)  #To tell how many unique entries

#This column has way too many unique entries to printed,so inorder to get meaningfulinsihts,
#We have to sort the value counts, either in ascending or descendng order.
#Since we are suspicious of the max value, we can sort in descending order.

#print(Autos['price'].value_counts().sort_index(ascending=False).head(50)) # This will give us the first x values in descending order & their corresponding frequencies
#print(Autos['price'].value_counts().sort_index().head(50)) #This will show us the x lowest values

#I cant really get much from the above, I will try another method.
#Since 75% of the prices are 7,200 or less and even the mean is 9,840. I want to find out how many values in this column are more than 50,000
#Also, we will check for prices less than 100 since it is not really realistic to sell a car for that price


#more_than_50 = Autos['price'] < 50000
less_than_100 = Autos['price'] < 100
#print(more_than_50.value_counts())
#print(less_than_100.value_counts())

#From the above code we can see that only about 200 entries out of 50,000 are above 50,000 euros.
#To avoid having an ununiform data, we can remove rows that have price greater than 50000

#Now we will remove rows with  prices that are less than 100 euros and greater than 50,000 euros from our dataframe

new_autos = Autos[Autos['price'].between(100,50000)]
#print(new_autos.shape)
#print(new_autos['price'].describe())

#Now, our max and min are 50,000 and 100 respectively

In [38]:
#Now we want to work with dates

#print(new_autos.info())

#print(new_autos[['date_crawled','last_seen','ad_created']].head()) # To expore the data in the date columns 
 #notice that the first 10 characters represent the day (e.g. 2016-03-12).
 #To understand the date range, we can extract just the date values, use Series.value_counts() to generate a distribution, and then sort by the index.

#For the date_crawled column

date_crawl = new_autos['date_crawled'].str[:10]  #This will create an array of the actual date for each entry

#To include missing values in the distribution & to use percentages instead of counts, chain the Series.value_counts(normalize=True, dropna=False) method.

#print(date_crawl.value_counts(normalize=True, dropna=False).sort_index())  #using normalize is true, gave me 0.0 for all so I will try normalize = False
#print(date_crawl.value_counts(dropna=False).sort_index()) # This returned value counts

#So I realized that the reason why the outputs of normalize =True gave me 0.0 was because I had earlier at the top of this notebook (line 4),I have formatted my numbers to 1 dp, so values like 0.03, 0.001 etc had to appear as 0.0
#I changed the format to 3 d.p and I got the desired outputs
#Upon exploring the "date_crawled" column, the adverts were crawled between march & early april of 2016

#For the 'last_seen' column

#last_seen_ = new_autos['last_seen'].str[:10]
#print(last_seen_.value_counts(normalize=True, dropna=False).sort_index())
#Upon exploring the "last_seen" column, the advertisers were last seen  between march & early april of 2016

#For the 'ad_created' column

#ad_created_ = new_autos['ad_created'].str[:10]
#print(ad_created_.value_counts(normalize=True, dropna=False).sort_index())
#Upon exploring the "ad_created" column, the ads were created as far back as june 2015 

#To investigate the 'registration_year'

#print(new_autos['registration_year'].describe())

#From reviewing the registration year, the minimum value of 1000 seems quite odd, Also max of 9999 is very wrong.
# Any car with registration date greater than 2016, should be removed
#Also, any car with registration year lesser than 1900 should be removed

#lets count the no of entries between 1900 and 2016

between_1900_2016 = (new_autos['registration_year'] > 1900) & (new_autos['registration_year'] < 2016)
#print(between_1900_2016.value_counts())

new_autos2 = new_autos[new_autos['registration_year'].between(1900,2016)]
#print(new_autos2.shape)

print(new_autos2['registration_year'].value_counts(normalize=True).sort_index())

1910   0.000
1927   0.000
1929   0.000
1931   0.000
1934   0.000
        ... 
2012   0.028
2013   0.017
2014   0.014
2015   0.008
2016   0.026
Name: registration_year, Length: 77, dtype: float64


In [39]:
# I want to create a dictionary that has the brand name as keys and the mean of the prices of all the cars of each brand as values

#First, let me decide on which brands to use

#print(new_autos2['brand'].value_counts(normalize=True))

#So I have decided on the ten brands with the highest frequncy of entries

brands_list = ['volkswagen','bmw','opel','mercedes_benz','audi','ford','peugeot','mazda','fiat','renault']
brands_avg_price={} #creatd an empty dict to store average price or each of the selected brands

for bra_nd in brands_list:
    this_brand = new_autos2[new_autos2['brand']== bra_nd]
    brand_average = this_brand['price'].mean()
    brands_avg_price[bra_nd]=brand_average
    
#print(brands_avg_price)

#I will create another dictionary with the average mileage for the selected brands

brand_avg_mileage = {}

for bra_nd in brands_list:
    this_brand = new_autos2[new_autos2['brand']== bra_nd]
    brand_average = this_brand['odometer_km'].mean()
    brand_avg_mileage[bra_nd]=brand_average

#print(brand_avg_mileage)

#I will now convert both dictionaries into a pandas series

series_avg_price = pd.Series(brands_avg_price)
#print(series_avg_price)

series_avg_miles = pd.Series(brand_avg_mileage)
#print(series_avg_miles)

#Now I will turn the avg price series to a dataframe

price_miles = pd.DataFrame(series_avg_price, columns= ['mean_price'])

#Now I will add the secnd series (with the mileage data) as another column to the dataframe which I just created

price_miles['mean_mileage'] = series_avg_miles

print(price_miles)

               mean_price  mean_mileage
volkswagen       5424.915    128823.619
bmw              8142.259    132953.073
opel             3005.496    129384.430
mercedes_benz    8233.911    131524.719
audi             9067.807    129780.000
ford             3649.642    124410.399
peugeot          3113.861    127127.890
mazda            4129.775    124553.824
fiat             2836.874    116950.295
renault          2454.574    128326.456
